In [1]:
'''
Skeleton code for resolution theorem prover for propositional logic

Yoonsuck Choe
10/20/2021

clause representation: 

  [<positive-proposition-list>, <negative-proposition-list>]

proposition-list representation: 

  [ 'p', 'q', 'r' ]

  Note: 'p', 'q', 'r', 's' are propositions : CASE SENSITIVE!

Example clause: 

  [['p', 'q'] ['r']] : This is clause P \/ Q \/ ~R 

'''

DEBUG=True

#---------------------------------------------------------------------
def rm_item(lst, item):  
#---------------------------------------------------------------------
  '''
  function: rm_pred()
  
  remove item from list
  - return [] if empty, not None
  '''
  lst.remove(item)

  if lst == None:
    return []
  else: 
    return lst


#---------------------------------------------------------------------
def mk_unique(clause):
#---------------------------------------------------------------------
  '''
  remove redundant propositions in the clause
  '''
  pos = clause[0]
  neg = clause[1]

  pos = list(set(pos))
  neg = list(set(neg))

  return [pos, neg]

#---------------------------------------------------------------------
def resolve(clause1, clause2):
#---------------------------------------------------------------------
  ''' 
  function: resolve() 

  [['p', 'q'] ['r']] : clause P \/ Q \/ ~R 

  resolve clauses c1 and c2 
  - if resolvable returns resolvent
  - if not resolvable returns False
  - returns empty clause [[], []] when Falsified

  '''

  # remove any redundant propositions in the clauses
  c1 = mk_unique(clause1)
  c2 = mk_unique(clause2)

  count = 0

  # print clauses
  if DEBUG:
    print("resolving: "+str(c1)+" and "+str(c2))

  # c1 pos vs. c2 neg
  for p1 in c1[0]:
    for p2 in c2[1]:
      if p1==p2:
        c1 = [rm_item(c1[0],p1), c1[1]]
        c2 = [c2[0], rm_item(c2[1],p2)]
        count = count+1

  # c2 pos vs. c1 neg
  for p1 in c1[1]:
    for p2 in c2[0]:
      if p1==p2:
        c1 = [c1[0], rm_item(c1[1],p1)]
        c2 = [rm_item(c2[0],p2), c2[1]]
        count = count+1

  # check for multiple matches or no match and abort if so.
  if count>1 or count==0:
    return False
  
  # compute resolvent
  c1[0].extend(c2[0])
  c1[1].extend(c2[1])

  # make unique and return
 
  return mk_unique(c1)



#---------------------------------------------------------------------
def null_p(clause):
#---------------------------------------------------------------------
  '''
  function null_p()

  check if "False" is derived 
  - returns True if empty clause
  - returns False if non-empty clause
  '''

  if (type(clause) is list) and len(clause[0]) + len(clause[1]) == 0:
    return True
  else:
    return False



#---------------------------------------------------------------------
def print_clause(clause):
#---------------------------------------------------------------------
  '''
  function print_clause

  See function resolve() for clause representation.

  '''

  s = ""
  for pos in clause[0]:
    s = s+pos+"\/"
  
  for neg in clause[1]:
    s = s+"~"+neg+"\/"
 
  if len(s)==0:
    print("NULL")
  else:
    n=len(s)
    print(s[0:(n-2)])

#------------------------------------------------------
# test resolve() function
#------------------------------------------------------

print("\ntest1")
test1 = resolve([['p', 'q'], ['r']], [[], ['p']])
print("resolvent = "+str(test1))
print("null? "+str(null_p(test1)))

print("\ntest2")
test2 = resolve([['p'], []], [[], ['p']])
print("resolvent = "+str(test2))
print("null? "+str(null_p(test2)))

print("\ntest3")
test3 = resolve([['p', 'q'], ['r']], [['r'], ['p']])
print("resolvent = "+str(test3))
print("null? "+str(null_p(test3)))

print("\ntest4")
test4 = resolve([['p', 'q'], ['r']], [['r'], []])
print("resolvent = "+str(test4))
print("null? "+str(null_p(test4)))


print("\ntest5")
test5 = resolve([['p', 'q'], ['r']], [['p', 'r'], []])
print("resolvent = "+str(test5))
print("null? "+str(null_p(test5)))


# edge case not being handled correctly: p \/ q \/ ~r \/ ~r  vs. p \/ r should give p \/ q, not p \/ q \/ ~r 
# - now this is fixed, with the use of mk_unique()
print("\ntest6")
test6 = resolve([['p', 'q'], ['r', 'r']], [['p', 'r'], []])
print("resolvent = "+str(test6))
print("null? "+str(null_p(test6)))



test1
resolving: [['q', 'p'], ['r']] and [[], ['p']]
resolvent = [['q'], ['r']]
null? False

test2
resolving: [['p'], []] and [[], ['p']]
resolvent = [[], []]
null? True

test3
resolving: [['q', 'p'], ['r']] and [['r'], ['p']]
resolvent = False
null? False

test4
resolving: [['q', 'p'], ['r']] and [['r'], []]
resolvent = [['q', 'p'], []]
null? False

test5
resolving: [['q', 'p'], ['r']] and [['p', 'r'], []]
resolvent = [['q', 'p'], []]
null? False

test6
resolving: [['q', 'p'], ['r']] and [['p', 'r'], []]
resolvent = [['q', 'p'], []]
null? False


In [5]:
from re import T
'''
Using the resolve() function and the null_p function, implement the two-pointer method for propositional logic theorem proving
- use set of support

theorem representation:

[ [<index> <clause1>], [<index> <clause2>], ... ]

index : integer 

clause representation:

[<positive-proposition-list>, <negative-proposition-list>]

proposition-list representation: 

[ 'p', 'q', 'r' ]

'''

# define your theorem, as a set of indexed clauses

theorem = [    # pos-list      # neg-list 
           [0, [['p','q'],    ['r']]],       # clause 0     P \/ Q \/ ~R
           [1, [['s'],        ['q']]],       # clause 1     S \/ ~Q 
           [2, [['t'],        ['s']]],       # clause 2     T \/ ~S 
           [3, [[],           ['p']]],       # clause 3     ~P   - negated conclusion starts here (goal clause index = 3)
           [4, [['r'],        []]],          # clause 4     R   
           [5, [[],           ['t']]]        # clause 5     ~T  
          ]

theorem2 = [ # pos-list      # neg-list 
           [0, [['I'],    ['M']]],       
           [1, [['M'],        ['I']]],       
           [2, [['M', 'L'],        []]],       
           [3, [['H'],           ['I']]],       
           [4, [['H'],        ['L']]],            
           [5, [['G'],           ['H']]],  
           [6, [[],           ['M']]] 
            ]   

theorem3 = [ # pos-list      # neg-list 
           [0, [['I'],    ['M']]],       
           [1, [['M'],        ['I']]],       
           [2, [['M', 'L'],        []]],       
           [3, [['H'],           ['I']]],       
           [4, [['H'],        ['L']]],            
           [5, [['G'],           ['H']]], 
           [6, [[],           ['G']]] 
            ]  # enter your own example for testing 

theorem4 = [ # pos-list      # neg-list 
           [0, [['I'],    ['M']]],       
           [1, [['M'],        ['I']]],       
           [2, [['M', 'L'],        []]],       
           [3, [['H'],           ['I']]],       
           [4, [['H'],        ['L']]],            
           [5, [['G'],           ['H']]],  
           [6, [[],           ['H']]] 
            ]  # enter your own example for testing 


#------------------------------------------------------
# prover() function : you must implement this function 
#---------------------------------------------------------------------
def prover(thm, goal):
#---------------------------------------------------------------------

  print('\nprover():\n\nTheorem:')
  outerPointer = goal
  while True:
    if outerPointer >= len(thm):
      return "failure"
    innerPointer = 0
    for innerPointer in range(outerPointer):
      resolvent = resolve(thm[innerPointer][1], thm[outerPointer][1])
      if null_p(resolvent): 
        return "success"
      if resolvent != False:
        identical = hasIdentical(resolvent, thm)
        if identical == False:
          thm.append([len(thm), resolvent])
          print_clause(resolvent)
    outerPointer = outerPointer + 1
  return " "

#------------------------------------------------------
# test prover() function
#------------------------------------------------------

def hasIdentical(resolvent, thm):
  for i in range(len(thm)):
    if len(thm[i][1][0]) != len(resolvent[0]):
      continue
    if len(thm[i][1][1]) != len(resolvent[1]):
      continue
    identical = True
    for j in range (len(thm[i][1][0])):
      if resolvent[0][j] != thm[i][1][0][j]:
        identical = False
        break
    if identical:
      for j in range (len(thm[i][1][1])):
        if resolvent[1][j] != thm[i][1][1][j]:
          identical = False
          break
    if identical:
      return True
  return False


# Test prover on example theorem

print("Result: " + prover(theorem, 3)) # default theorem
print("Result: " + prover(theorem2, 6)) # Unicorn theorem M
print("Result: " + prover(theorem3, 6)) # Unicorn theorem G
print("Result: " + prover(theorem4, 6)) # Unicorn theorem H


prover():

Theorem:
resolving: [['q', 'p'], ['r']] and [[], ['p']]
q\/~r
resolving: [['s'], ['q']] and [[], ['p']]
resolving: [['t'], ['s']] and [[], ['p']]
resolving: [['q', 'p'], ['r']] and [['r'], []]
q\/p
resolving: [['s'], ['q']] and [['r'], []]
resolving: [['t'], ['s']] and [['r'], []]
resolving: [[], ['p']] and [['r'], []]
resolving: [['q', 'p'], ['r']] and [[], ['t']]
resolving: [['s'], ['q']] and [[], ['t']]
resolving: [['t'], ['s']] and [[], ['t']]
~s
resolving: [[], ['p']] and [[], ['t']]
resolving: [['r'], []] and [[], ['t']]
resolving: [['q', 'p'], ['r']] and [['q'], ['r']]
resolving: [['s'], ['q']] and [['q'], ['r']]
s\/~r
resolving: [['t'], ['s']] and [['q'], ['r']]
resolving: [[], ['p']] and [['q'], ['r']]
resolving: [['r'], []] and [['q'], ['r']]
q
resolving: [[], ['t']] and [['q'], ['r']]
resolving: [['q', 'p'], ['r']] and [['q', 'p'], []]
resolving: [['s'], ['q']] and [['q', 'p'], []]
s\/p
resolving: [['t'], ['s']] and [['q', 'p'], []]
resolving: [[], ['p']] and [['